# NETCONF实例
* 实例：BGP配置快速解析

In [1]:
import xmltodict
from ncclient import manager
from helper import bgpfunc as bf

iosXE_netconf_info = {"host":"192.168.1.61",
                      "port": 830,
                      "username":"netconf",
                      "password":"netconf123",
                      "hostkey_verify":False}

## 创建初始化NETCONF连接

In [2]:
def create_netconf_connection(iosXE_netconf_info):
    session = manager.connect(**iosXE_netconf_info)
    return session

## BGP 配置快速解析
* 面对复杂BGP网络时快速了解网络配置情况
* 存在vrf的BGP配置
* 不存在vrf的BGP配置

In [3]:
def with_vrf(config):
    if config.get('address-family'):
        # Find vrfs, if any
        vrf_config = config['address-family']['with-vrf']['ipv4']['vrf']
        if type(vrf_config) != list:
            vrf_config = [vrf_config]
        for one_vrf_config in vrf_config:
            af = one_vrf_config['ipv4-unicast']
            vrf_name = one_vrf_config['name']

            # Begin configuration for BGP perameters
            if af.get('neighbor'):
                print(f"\nAddress Family: {vrf_name} -------")
                print(f"\n   AF Neighbor Information:\n")
                bf.address_family(af.get('neighbor'))
            if af.get("redistribute"):
                print(f"\n   AF Redistribution:\n")
                [bf.redistribution(k, v) for k, v in af.get("redistribute").items()]
            if af.get('network'):
                print(f"\n   AF Network Statements:\n")
                bf.networks(af.get('network', {}))
            if af.get('prefix-list'):
                bf.networks(af.get('prefix-list', {}))

#### Legacy routing config, non address-family

In [4]:
def legacy(config):
    if config.get('neighbor'):
        print(f"   Neighbors -------------\n")
        list(map(bf.neighbor, config.get('neighbor', {})))
    if config.get('network'):
        print(f"\n   Network Statements-----------------------\n")
        list(map(bf.networks, config.get('network', {})))
    if config.get("redistribute"):
        print(f"\n   Redistribution -----------------------\n")
        [bf.redistribution(k, v) for k, v in config.get("redistribute", {}).items()]
    if config.get('prefix-list'):
        list(map(bf.networks, config.get('prefix-list', {})))

#### Parse and prints router BGP configuration

In [5]:
def search_config(config):
    # Get legacy routing configuration
    print(f"\n--------------------- Local AS: {config.get('id')} ---------------------\n")

    if config.get('bgp', {}).get('log-neighbor-changes', {}):
        print(f"Logging Neighbor: {config.get('bgp').get('log-neighbor-changes')}\n")

    legacy(config)
    with_vrf(config)

### 开始获取数据

In [6]:
get_policies = """
<filter xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
    <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
        <router/>
    </native>
</filter>"""

session = create_netconf_connection(iosXE_netconf_info)
config_data = session.get(get_policies)

In [7]:
router_data = xmltodict.parse(config_data.xml)["rpc-reply"]["data"]

* [关于OrderedDict](http://www.mamicode.com/info-detail-2582729.html)

### 筛选BGP相关配置

In [8]:
bgp_details = router_data["native"].get("router", {}).get("bgp", {})

### 对BGP配置进行搜索解析

In [9]:
search_config(bgp_details)


--------------------- Local AS: 45000 ---------------------

Logging Neighbor: true

   Neighbors -------------

         Remote AS: 40000
          Neighbor: 1.1.1.100
     Next-Hop-Self: No
         Route-Map: None           Direction: None
       Prefix-list: None           Direction: None
   Route-Reflector: No
     Soft-Reconfig: No


         Remote AS: 50000
          Neighbor: 2.2.2.100
     Next-Hop-Self: No
         Route-Map: None           Direction: None
       Prefix-list: None           Direction: None
   Route-Reflector: No
     Soft-Reconfig: No



Address Family: vpn1 -------

   AF Neighbor Information:

          Neighbor: 5.5.5.100 
     Next-Hop-Self: No
   Route-Reflector: No
         Route-Map: None           Direction: None
       Prefix-list: None           Direction: None
          Activate: Yes


   AF Network Statements:

            Network:       None  Mask: None

Address Family: vpn2 -------

   AF Neighbor Information:

          Neighbor: 5.5.5.101 
 